In [1]:
import tensorflow_federated as tff
import tensorflow as tf
import tensorflow_text as text
import collections
from IPython.display import display
from tensorflow import keras
import pandas as pd
from random import sample
import numpy as np
# import seaborn as sns
import os
from tqdm import trange
from random import sample
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense,LSTM,Dropout,Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
# import tensorflow_text as text
import tensorflow_hub as hub
# from tensorflow.keras.losses import MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,MeanAbsolutePercentageError
# from tensorflow.keras.metrics import MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,MeanAbsolutePercentageError
import nest_asyncio
from time import time
import sys

import uuid



2023-05-23 09:55:02.377447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 09:55:03.163947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Found {device_count} GPU(s) available.")
    for i in range(device_count):
        device_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {device_name}")
else:
    print("No GPUs available.")


Found 1 GPU(s) available.
GPU 0: Tesla T4


In [3]:
tf.config.list_physical_devices('GPU')




2023-05-23 09:55:07.250171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-23 09:55:07.253195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-23 09:55:07.255185: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
def get_training_Q(Q):
    size = int(Q*len(train_data.client_ids))
    sampled_clients = np.random.choice(
            train_data.client_ids,
            size=size,
            replace=False)
    
    sampled_train_data = [
            train_data.create_tf_dataset_for_client(client)
            for client in sampled_clients
        ]
    
    return sampled_train_data

def get_test_Q(Q):
    size = int(Q*len(test_data.client_ids))
    sampled_clients = np.random.choice(
            test_data.client_ids,
            size=size,
            replace=False)
    
    ids = [_id.split('/')[-3] for _id in sampled_clients]
    print(ids)
    
    sampled_test_data = [
            test_data.create_tf_dataset_for_client(client)
            for client in sampled_clients
        ]
    return sampled_test_data

def get_eval_dataset():
    test_data_paths = glob('../Dataset/*/test/*/scaled/')
    
    dataset_test= []
    for path in test_data_paths:
        
        d = dict(X= np.load(f'{path}data_X.npy'),
                 y= np.load(f'{path}data_Y.npy'))    
        dataset_test.append(d)
        
    dataset = {}
    
    dataset['X_test'] = np.concatenate([data['X'] for data in dataset_test],axis=0)
    dataset['y_test'] = np.concatenate([data['y'] for data in dataset_test],axis=0)
    
    return dataset

def evaluate_model(model):
    X_test= eval_dataset['X_test']
    y_test = eval_dataset['y_test']
    
    ev = model.evaluate(x=X_test,y=y_test,batch_size=2048,return_dict=True)
    result  = pd.DataFrame(ev,index=[0])
    return result

def from_federated_to_keras(state):
    federated_model = base_model()
    federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1.0),
                            loss=tf.keras.losses.MeanSquaredError(),
                            metrics=[MeanAbsoluteError(), MeanSquaredError(), RootMeanSquaredError(),
                                    MeanAbsolutePercentageError() 
                                    ])
    state.model.assign_weights_to(model=federated_model)
    return federated_model
    
def evaluate_federated_model(state):
    federated_model = from_federated_to_keras(state)

    return evaluate_model(federated_model)

def get_model_in_text(path,model_fn):
    m = model_fn()
    with open(f'{path}model_report.txt','w') as fh:
        m.summary(print_fn=lambda x: fh.write(x + '\n'))

def save_all(results,state,n_rounds,config,unique_id):
    results.reset_index(inplace=True,drop=True)

    centralized_model = from_federated_to_keras(state=state)
    path = f'/scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/{unique_id}/{n_rounds}/'

    config['round'] = n_rounds
    

    if not os.path.exists(path):
        print(f'Saving in {path}')
        os.makedirs(path)
        pd.DataFrame(config,index=[0]).to_csv(f'{path}config.csv')
        get_model_in_text(path,base_model)
        results.to_csv(f'{path}results.csv')
        centralized_model.save(f'{path}model')

In [19]:
#GET-DATASETS
from glob import glob
from random import sample
import pandas as pd
import tensorflow as tf
import collections
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPAdamGaussianOptimizer
tf.config.run_functions_eagerly(True)


# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')  # load pretrained tokenizer model

# PREFETCH_BUFFER = 512
# BATCH_SIZE = 512
# NUM_EPOCHS = 10
PREFETCH_BUFFER = 1024
BATCH_SIZE = 4
NUM_EPOCHS = 5

df = pd.read_csv("../../../models/firstTry/df_testing.csv").sample(1000)
tokens = set()
for message in df["x"]:
    text_tokens = message.split()  # Split text into tokens based on spaces or other delimiters
    tokens.update(text_tokens)

_VOCAB = list(set(tokens))

_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=_VOCAB,
        key_dtype=tf.string,
        values=tf.range(tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
        value_dtype=tf.int64
    ),
    num_oov_buckets=1
)

tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)

text_tokens = tokenizer.tokenize(df["x"])

def create_tf_dataset_for_client_fn(federation_level):
    max_length = 512

    def tokenize_map_fn(data):
        tokenized_inputs = tokenizer.tokenize(data['x'])
        input_ids = lookup_table.lookup(tokenized_inputs)
        input_ids = input_ids.to_tensor()
        return collections.OrderedDict(x=input_ids, y=data["y"])

    # def tf_tokenize_map_fn(data):
    #     # print(data)
    #     input_ids, label = tokenize_map_fn(data['x'], data["y"])
    #     # input_ids.set_shape([None])
    #     # label.set_shape([])
    #     return 

    df_client = df[df["federation_level"] == federation_level]
    data_x = df_client.x.values
    data_y = df_client.y.values
    data = collections.OrderedDict((('y', data_y), ('x', data_x)))
    dataset = tf.data.Dataset.from_tensor_slices(data)
    # print(dataset)
    dataset = dataset.map(tokenize_map_fn)
    
    # ordered_dict_dataset = dataset.map(lambda x: collections.OrderedDict([('x', x['input_ids']), ('y', x['label'])]))

    return dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)


train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=df.federation_level.unique().tolist(),
        serializable_dataset_fn=create_tf_dataset_for_client_fn
    )

test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=df.federation_level.unique().tolist(),
        serializable_dataset_fn=create_tf_dataset_for_client_fn
    )

example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

train_data = [train_data.create_tf_dataset_for_client(client_id) for client_id in train_data.client_ids]


/home/ubuntu/federated/venv/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [20]:
train_data[0].element_spec

OrderedDict([('x',
              TensorSpec(shape=(None, None, None, None), dtype=tf.int64, name=None)),
             ('y', TensorSpec(shape=(None,), dtype=tf.int64, name=None))])

In [21]:
import tensorflow as tf
import tensorflow_federated as tff
# from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError, RootMeanSquaredError, MeanAbsolutePercentageError

class BertTextClassifier(tf.keras.Model):
    def __init__(self, num_labels, vocab_size, embedding_dim, dropout=0.5):
        super(BertTextClassifier, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=512)
        self.dropout = Dropout(dropout)
        self.dense = Dense(768, activation='relu')
        self.final_dense = Dense(num_labels, activation='softmax')

    def call(self, inputs):
        # print(inputs)
        embedded_input = self.embedding(inputs)  # Update the key to "input_ids"
        pooled_output = tf.reduce_mean(embedded_input, axis=1)
        dropout_output = self.dropout(pooled_output)
        dense_output = self.dense(dropout_output)
        final_output = self.final_dense(dense_output)
        return final_output



def model_tff():
    model = BertTextClassifier(num_labels=len(df.y.unique()), vocab_size=_VOCAB_SIZE, embedding_dim=256)  # Replace your_num_labels with the actual number of labels
    return tff.learning.models.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])



In [22]:
def get_iterative_process(noise, n_clients=0):
    if noise == 0.0:
        iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
            model_fn=model_tff,
            client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
            server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1),
            use_experimental_simulation_loop=True
        )
    else:
        aggregation_factory = tff.learning.model_update_aggregator.dp_aggregator(noise, n_clients)
        iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
            model_fn=model_tff,
            client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
            server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1),
            # model_update_aggregation_factory=aggregation_factory
        )
    return iterative_process

Noise = 0.5
total_clients = len(df.federation_level.unique())
n_clients = len(df.federation_level.unique())
tf.config.run_functions_eagerly(False)



iterative_process = get_iterative_process(Noise, n_clients)
state = iterative_process.initialize()
state, metrics = iterative_process.next(state, train_data)

# for round_num in range(4):
#     state, metrics = iterative_process.next(state, train_data)
#     print(f"Round {round_num+1} - Train Loss: {metrics['loss']}")


2023-05-23 10:12:49.657544: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_1' with dtype float and shape [?,10]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_1}}]]
2023-05-23 10:12:49.657653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_4' with dtype float and shape [?,?,?,768]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_4}}]]
2023-05-23 10:12:49.657704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) E

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Failed to run computation: Op type not registered 'RegexSplitWithOffsets' in binary running on kilian-tf. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
	 [[{{node MapDataset/_9}}]]
	 [[root/1/Placeholder/DatasetFromGraph]] while evaluating local [_ovm25] in block locals [_ovm3,_ovm4,_ovm5,_ovm6,_ovm20,_ovm21,_ovm22,_ovm23,_ovm24,_ovm25,_ovm26,_ovm27,_ovm28,_ovm53,_ovm54,_ovm55,_ovm56,_ovm57,_ovm58,_ovm59,_ovm60,_ovm61,_ovm62,_ovm63,_ovm64,_ovm65,_ovm66,_ovm67,_ovm68,_ovm69,_ovm78,_ovm79,_ovm80,_ovm81,_ovm82,_ovm83,_ovm84,_ovm85,_ovm86,_ovm87,_ovm88,_ovm89]"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:

In [ ]:
Q = n_clients/total_clients
config = {'Q':Q,
          'clients_per_round':int(n_clients),
          'total_clients':total_clients,
          'noise': Noise,
          'batch_size':int(BATCH_SIZE),
          'eval_client_size':len(glob('../Dataset/*/test/*/scaled/')),
          'internal_epochs':int(NUM_EPOCHS)}

In [156]:
nest_asyncio.apply()
state = iterative_process.initialize()  
NUM_ROUNDS = 125
results = pd.DataFrame()
unique_id = str(uuid.uuid4())[0:8]


In [8]:
sampled_train_data = [train_data.create_tf_dataset_for_client(client_id) for client_id in train_data.client_ids]
state, metrics = iterative_process.next(state, sampled_train_data)


2023-05-22 16:01:00.581902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-22 16:01:00.583379: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-22 16:01:00.583524: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-22 16:01:00.583852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-22 16:01:00.585016: W tensorflow/core/common_runtime/gpu/gpu_d

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Failed to run computation: No OpKernel was registered to support Op 'EagerPyFunc' used by {{node EagerPyFunc}} with these attrs: [Tout=[DT_INT32, DT_INT32], Tin=[DT_STRING, DT_INT64], token="pyfunc_3", is_async=false]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[EagerPyFunc]]
	 [[StatefulPartitionedCall/ReduceDataset]] while evaluating local [_ern25] in block locals [_ern3,_ern4,_ern5,_ern6,_ern20,_ern21,_ern22,_ern23,_ern24,_ern25,_ern26,_ern27,_ern28,_ern53,_ern54,_ern55,_ern56,_ern57,_ern58,_ern59,_ern60,_ern61,_ern62,_ern63,_ern64,_ern65,_ern66,_ern67,_ern68,_ern69,_ern78,_ern79,_ern80,_ern81,_ern82,_ern83,_ern84,_ern85,_ern86,_ern87,_ern88,_ern89]"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:37639 {created_time:"2023-05-22T16:01:01.031072298+00:00", grpc_status:13, grpc_message:"Failed to run computation: No OpKerne

In [18]:
t = trange(1,NUM_ROUNDS, desc='', leave=True)
        
for round_num in t:
    

    start = time()
    
    train_set = get_training_Q(Q)

    state, metrics = iterative_process.next(state, train_set)
    
    if round_num % 25 == 0:
        metrics['train']['round_number'] = int(round_num)
        metrics['train']['time'] = time()-start
        metrics_train_df = pd.DataFrame(metrics['train'],index=[0])
        sys.stdout.flush()
        
        if Noise != 0.0:
            print(state.delta_aggregate_state)

        print(f'Train:{metrics_train_df["mean_absolute_percentage_error"].iloc[0]}')
        
        metrics_test_df = evaluate_federated_model(state)
        metrics_test_df['round_number']= round_num
        metrics_train_df = metrics_train_df.merge(metrics_test_df, on='round_number', suffixes=('_train','_test'))
        results = pd.concat([results,metrics_train_df])
       
        save_all(results=results,state= state,n_rounds=round_num,config=config,unique_id=unique_id)
       
        

  0%|          | 0/124 [00:00<?, ?it/s]2023-04-18 16:59:52.451641: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-18 16:59:52.899884: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
 19%|█▉        | 24/124 [55:26<3:23:05, 121.86s/it]

OrderedDict([('zeroing_norm', (_GlobalState(current_estimate=3.1622777, target_quantile=0.98, learning_rate=2.3025851, below_estimate_state=()), ())), ('inner_agg', (_GlobalState(numerator_state=_GlobalState(noise_multiplier=0.48113248, sum_state=_GlobalState(l2_norm_clip=0.4943598, stddev=0.23785256), quantile_estimator_state=_GlobalState(current_estimate=0.4943598, target_quantile=0.5, learning_rate=0.2, below_estimate_state=_GlobalState(numerator_state=_GlobalState(l2_norm_clip=0.5, stddev=3.5), denominator=70.0))), denominator=70.0), ())), ('zeroed_count_agg', ())])
Train:55.05063247680664


2023-04-18 17:53:42.278568: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-18 17:53:43.475120: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8004


8453/8453 [==============================] - 27s 3ms/step - loss: 2.9891e-04 - mean_absolute_error: 0.0099 - mean_squared_error: 2.9891e-04 - root_mean_squared_error: 0.0173 - mean_absolute_percentage_error: 69.8990
Saving in /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/25/


2023-04-18 17:54:10.968084: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


...

INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/75/model/assets


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/75/model/assets
 80%|███████▉  | 99/124 [3:37:37<53:38, 128.73s/it]  

OrderedDict([('zeroing_norm', (_GlobalState(current_estimate=0.100000046, target_quantile=0.98, learning_rate=2.3025851, below_estimate_state=()), ())), ('inner_agg', (_GlobalState(numerator_state=_GlobalState(noise_multiplier=0.48113248, sum_state=_GlobalState(l2_norm_clip=0.25863373, stddev=0.12443709), quantile_estimator_state=_GlobalState(current_estimate=0.25863373, target_quantile=0.5, learning_rate=0.2, below_estimate_state=_GlobalState(numerator_state=_GlobalState(l2_norm_clip=0.5, stddev=3.5), denominator=70.0))), denominator=70.0), ())), ('zeroed_count_agg', ())])
Train:34.83448028564453
8453/8453 [==============================] - 24s 3ms/step - loss: 1.5793e-04 - mean_absolute_error: 0.0067 - mean_squared_error: 1.5793e-04 - root_mean_squared_error: 0.0126 - mean_absolute_percentage_error: 45.2697
Saving in /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/model/assets


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/model/assets
100%|██████████| 124/124 [4:32:03<00:00, 129.64s/it]


In [19]:
#  MAPE = 25.108067
# Q	0.005794
# clients_per_round	10
# total_clients	1726
# noise	0.10
# batch_size	512
# internal_epochs	10
# Optimizer ADAM
# 100 rounds 

In [20]:
results

,mean_absolute_error_train,mean_squared_error_train,root_mean_squared_error_train,mean_absolute_percentage_error_train,loss_train,round_number,time,loss_test,mean_absolute_error_test,mean_squared_error_test,root_mean_squared_error_test,mean_absolute_percentage_error_test
0,0.006792,0.000192,0.013864,55.050632,0.000192,25,132.741692,0.000299,0.009923,0.000299,0.017289,69.899033
1,0.005065,0.000095,0.009732,34.695339,0.000095,50,124.682097,0.000218,0.008156,0.000218,0.014752,55.447880
2,0.005063,0.000090,0.009490,33.979881,0.000090,75,121.416200,0.000187,0.007681,0.000187,0.013659,56.774860
3,0.004827,0.000083,0.009105,34.834480,0.000083,100,120.216939,0.000158,0.006653,0.000158,0.012567,45.269745
